In [1]:
from ssd import build_ssd

## box_utils裡面很多Function，可以看看是怎麼設計的
from layers.box_utils import * 
from torch.autograd import Variable
from layers import functions
from layers import modules
from math import sqrt 
from itertools import product
from torch.autograd import Function

import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import argparse
import pickle
import torch.nn.functional as F


In [2]:
## 詳細模型結構可以參考ssd.py
ssd_net=build_ssd('train', size=300, num_classes=21)

C:\Users\user\Desktop\python\深度學習100天\Day029-31\Object Detection 程式導讀\ssd.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  self.priors = Variable(self.priorbox.forward(), volatile=True)


## 默認Config檔案在data/config.py內

In [3]:
ssd_net.cfg

{'num_classes': 21,
 'lr_steps': (80000, 100000, 120000),
 'max_iter': 120000,
 'feature_maps': [38, 19, 10, 5, 3, 1],
 'min_dim': 300,
 'steps': [8, 16, 32, 64, 100, 300],
 'min_sizes': [30, 60, 111, 162, 213, 264],
 'max_sizes': [60, 111, 162, 213, 264, 315],
 'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
 'variance': [0.1, 0.2],
 'clip': True,
 'name': 'VOC'}

In [4]:
cfg = {
    'num_classes': 21,
    'lr_steps': (80000, 100000, 120000),
    'max_iter': 120000,
    'feature_maps': [38, 19, 10, 5, 3, 1],
    'min_dim': 300,
    'steps': [8, 16, 32, 64, 100, 300],
    'min_sizes': [30, 60, 111, 162, 213, 264],
    'max_sizes': [60, 111, 162, 213, 264, 315],
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
    'variance': [0.1, 0.2],
    'clip': True,
    'name': 'VOC',
}

### 'aspect_ratios' : 使用六張Feature Map，每一張上方有預設的anchor boxes，Boxes aspect ratio可以自己設定
### 'feature_maps' : 使用feature map大小為[38x38, 19x19, 10x10, 5x5, 3x3, 1x1]
### 'min_sizes'、'max_sizes'可藉由下方算式算出，由作者自行設計
### 'steps' : Feature map回放回原本300*300的比例，如38要回放為300大概就是8倍
### 'variance' : Training 的一個trick，加速收斂，詳見：https://github.com/rykov8/ssd_keras/issues/53

---

## 'min_sizes'、'max_sizes' 計算

In [5]:
import math
## source:https://blog.csdn.net/gbyy42299/article/details/81235891
min_dim = 300   ## 维度
# conv4_3 ==> 38 x 38
# fc7 ==> 19 x 19
# conv6_2 ==> 10 x 10
# conv7_2 ==> 5 x 5
# conv8_2 ==> 3 x 3
# conv9_2 ==> 1 x 1
mbox_source_layers = ['conv4_3', 'fc7', 'conv6_2', 'conv7_2', 'conv8_2', 'conv9_2'] ## prior_box來源層，可以更改。很多改進都是基於此處的調整。
# in percent %
min_ratio = 20 ## 這裡即是論文中所說的Smin的= 0.2，Smax的= 0.9的初始值，經過下面的運算即可得到min_sizes，max_sizes。
max_ratio = 90
step = int(math.floor((max_ratio - min_ratio) / (len(mbox_source_layers) - 2)))## 取一個間距步長，即在下面用於循環給比取值時起一個間距作用。可以用一個具體的數值代替，這裡等於17。
min_sizes = []  ## 經過以下運算得到min_sizes和max_sizes。
max_sizes = []
for ratio in range(min_ratio, max_ratio + 1, step):
    ## 從min_ratio至max_ratio + 1每隔步驟= 17取一個值賦值給比。注意範圍函數的作用。
    ## min_sizes.append（）函數即把括號內部每次得到的值依次給了min_sizes。
    min_sizes.append(min_dim * ratio / 100.)
    max_sizes.append(min_dim * (ratio + step) / 100.)
min_sizes = [min_dim * 10 / 100.] + min_sizes
max_sizes = [min_dim * 20 / 100.] + max_sizes

## steps: 這一步要仔細理解，即計算卷積層產生的prior_box距離原圖的步長，先驗框中心點的坐標會乘以step，
## 相當於從特徵映射位置映射回原圖位置，比如conv4_3輸出特徵圖大小為38 *38，而輸入的圖片為300* 300，
## 所以38 *8約等於300，所以映射步長為8.這是針對300* 300的訓練圖片。
steps = [8, 16, 32, 64, 100, 300]  
aspect_ratios = [[2], [2, 3], [2, 3], [2, 3], [2], [2]]
 
print('min_sizes: ',min_sizes)
print('max_sizes: ',max_sizes)


min_sizes:  [30.0, 60.0, 111.0, 162.0, 213.0, 264.0]
max_sizes:  [60.0, 111.0, 162.0, 213.0, 264.0, 315.0]


---

## Default anchor boxes設計原理，看懂收穫很多
##### 可以理解 SSD原文中 8732個anchors是怎麼來的
##### 38×38×4+19×19×6+10×10×6+5×5×6+3×3×4+1×1×4=8732

In [6]:
class PriorBox(object):
    """Compute priorbox coordinates in center-offset form for each source
    feature map.
    """
    def __init__(self, cfg):
        super(PriorBox, self).__init__()
        self.image_size = cfg['min_dim']
        # number of priors for feature map location (either 4 or 6)
        self.num_priors = len(cfg['aspect_ratios'])
        self.variance = cfg['variance'] or [0.1]
        self.feature_maps = cfg['feature_maps']
        self.min_sizes = cfg['min_sizes']
        self.max_sizes = cfg['max_sizes']
        self.steps = cfg['steps']
        self.aspect_ratios = cfg['aspect_ratios']
        self.clip = cfg['clip']
        self.version = cfg['name']
        for v in self.variance:
            if v <= 0:
                raise ValueError('Variances must be greater than 0')

    def forward(self):
        mean = []
        '''依照Feature map大小找出所有的pixel 中心'''
        '''下方這兩個loop會找出W個x軸pixel對上W個y軸pixel，假如現在是在38x38的feature map上，就會有38x38個值'''
        '''ex. [0,1],[0,2]..[0,37] [1,1],[1,2]..[1,37]..........[37,37]'''
        for k, f in enumerate(self.feature_maps):
            for i, j in product(range(f), repeat=2):
                f_k = self.image_size / self.steps[k] ## 如self.steps==8，就是先將原圖size normalize(/300)後再乘上8
                # unit center x,y
                '''中心點'''
                cx = (j + 0.5) / f_k
                cy = (i + 0.5) / f_k

                # aspect_ratio: 1
                # rel size: min_size
                '''/self.image_size 就是在做normalization '''
                s_k = self.min_sizes[k]/self.image_size
                '''小的正方形box'''
                mean += [cx, cy, s_k, s_k]

                # aspect_ratio: 1
                # rel size: sqrt(s_k * s_(k+1))
                '''大的正方形box'''
                s_k_prime = sqrt(s_k * (self.max_sizes[k]/self.image_size))
                mean += [cx, cy, s_k_prime, s_k_prime]

                # rest of aspect ratios
                for ar in self.aspect_ratios[k]:
                    '''aspect ratio 2,3'''
                    mean += [cx, cy, s_k*sqrt(ar), s_k/sqrt(ar)]
                    '''aspect ratio 1/2,1/3'''
                    mean += [cx, cy, s_k/sqrt(ar), s_k*sqrt(ar)]
        # back to torch land
        output = torch.Tensor(mean).view(-1, 4)
        if self.clip:
            output.clamp_(max=1, min=0)
        return output

In [7]:
PriorBox_Demo=PriorBox(cfg)

In [8]:
print(PriorBox_Demo.forward().shape)

torch.Size([8732, 4])


---

## Loss 如何設計

In [9]:
class MultiBoxLoss(nn.Module):

    def __init__(self, num_classes, overlap_thresh, prior_for_matching,
                 bkg_label, neg_mining, neg_pos, neg_overlap, encode_target,
                 use_gpu=True):
        super(MultiBoxLoss, self).__init__()
        self.use_gpu = use_gpu
        '''有幾類'''
        self.num_classes = num_classes
        '''判定為正樣本的threshold，一般設為0.5'''
        self.threshold = overlap_thresh
        '''background自己會有一類，不用Label，假如我們有20類一樣標註0-19，下方會自己空出一類給background'''
        self.background_label = bkg_label
        self.encode_target = encode_target
        self.use_prior_for_matching = prior_for_matching
        '''OHEM，找出分得最不好的樣品，也就是confidence score比較低的正負樣品'''
        self.do_neg_mining = neg_mining
        '''負樣品與正樣品的比例，通常是3:1'''
        self.negpos_ratio = neg_pos
        self.neg_overlap = neg_overlap
        self.variance = cfg['variance']
     

    def forward(self, predictions, targets):

        '''prediction會output三個值'''
        '''loc shape: bounding box 資訊，torch.size(batch_size,num_priors,4)'''
        '''conf shape: 每一個bounding box 的信心程度，torch.size(batch_size,num_priors,num_classes)'''
        '''priors shape: 預設的defaul box， torch.size(num_priors,4)'''
        loc_data, conf_data, priors = predictions
        num = loc_data.size(0)
        priors = priors[:loc_data.size(1), :]
        num_priors = (priors.size(0))
        num_classes = self.num_classes

        # match priors (default boxes) and ground truth boxes
        loc_t = torch.Tensor(num, num_priors, 4)
        conf_t = torch.LongTensor(num, num_priors)
        for idx in range(num):
            truths = targets[idx][:, :-1].data
            labels = targets[idx][:, -1].data
            defaults = priors.data
            '''jaccard 計算每一個BBOX與ground truth的IOU'''
            match(self.threshold, truths, defaults, self.variance, labels,
                  loc_t, conf_t, idx)
        if self.use_gpu:
            loc_t = loc_t.cuda()
            conf_t = conf_t.cuda()
        '''用Variable包裝'''
        loc_t = Variable(loc_t, requires_grad=False)
        conf_t = Variable(conf_t, requires_grad=False)

        pos = conf_t > 0
        num_pos = pos.sum(dim=1, keepdim=True)


        pos_idx = pos.unsqueeze(pos.dim()).expand_as(loc_data)
        loc_p = loc_data[pos_idx].view(-1, 4)
        loc_t = loc_t[pos_idx].view(-1, 4)
        '''smooth_l1_loss 計算bounding box regression'''
        loss_l = F.smooth_l1_loss(loc_p, loc_t, size_average=False)

        # Compute max conf across batch for hard negative mining
        batch_conf = conf_data.view(-1, self.num_classes)
        loss_c = log_sum_exp(batch_conf) - batch_conf.gather(1, conf_t.view(-1, 1))

        # Hard Negative Mining
        loss_c = loss_c.view(num, -1)
        loss_c[pos] = 0
        '''排列confidence 的分數'''
        _, loss_idx = loss_c.sort(1, descending=True)
        _, idx_rank = loss_idx.sort(1)
        num_pos = pos.long().sum(1, keepdim=True)
        '''負樣品取出數量 == negpos_ratio*num_pos'''
        num_neg = torch.clamp(self.negpos_ratio*num_pos, max=pos.size(1)-1)
        neg = idx_rank < num_neg.expand_as(idx_rank)

        # Confidence Loss Including Positive and Negative Examples
        pos_idx = pos.unsqueeze(2).expand_as(conf_data)
        neg_idx = neg.unsqueeze(2).expand_as(conf_data)
        conf_p = conf_data[(pos_idx+neg_idx).gt(0)].view(-1, self.num_classes)
        targets_weighted = conf_t[(pos+neg).gt(0)]
        '''用cross_entropy做分類'''
        loss_c = F.cross_entropy(conf_p, targets_weighted, size_average=False)

        # Sum of losses: L(x,c,l,g) = (Lconf(x, c) + αLloc(x,l,g)) / N
        #double轉成torch.float64
        N = num_pos.data.sum().double()
        loss_l = loss_l.double()
        loss_c = loss_c.double()
        loss_l /= N
        loss_c /= N
        return loss_l, loss_c


## 產生我們Loss function，注意這裡的class要包含背景

In [10]:
Use_cuda=False
criterion = MultiBoxLoss(21, 0.5, True, 0, False, 3, 0.5,False, Use_cuda,)

----

## 基本設定

In [11]:
ssd_net=build_ssd('train', size=300, num_classes=21)
use_pretrained=False
if use_pretrained:
    ssd_net.load_weights('./demo/ssd300_mAP_77.43_v2.pth')
net=ssd_net

In [12]:
'''要不要使用gpu'''
Use_cuda=False

'''tensor type會依照cpu或gpu有所不同'''
if torch.cuda.is_available():
    if Use_cuda:
        torch.set_default_tensor_type('torch.cuda.FloatTensor')
    else:
        print("WARNING: It looks like you have a CUDA device, but aren't " +
              "using CUDA.\nRun with --cuda for optimal training speed.")
        torch.set_default_tensor_type('torch.FloatTensor')
else:
    torch.set_default_tensor_type('torch.FloatTensor')


'''使用GPU時模型要轉成cuda'''
if Use_cuda:
    net = net.cuda()
    
batch_size_=32
optimizer = optim.Adam(net.parameters(),lr=0.00001/batch_size_)

Run with --cuda for optimal training speed.


---

## 訓練

## 這裡我們先示範輸入的 image,Label格式，真正在訓練時，準備成一樣格式即可

In [13]:
'''輸入影像格式，假設batch size 為 4'''
image_in=torch.tensor(torch.rand(4,3,300,300),dtype=torch.float32)
'''Label格式，沒有固定長度，看圖像中有幾個label就有幾個'''
label_0=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_1=[[ 0.1804,  0.6076,  0.7701,  0.8485, 13.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 11.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 7.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 5.0000],]
label_2=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 14.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]
label_3=[[ 0.1804,  0.6076,  0.7701,  0.8485, 0.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 3.0000],
       [ 0.2250,  0.0000,  0.9238,  0.5641, 19.0000],
       [ 0.2950,  0.0000,  0.8238,  0.3641, 6.0000],]

C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


In [14]:
epochs=300
iteration=1000

In [ ]:
for epoch in range(epochs):
    n=0
    loss_sum=[]
    loc_loss=[]
    conf_loss=[]
    for number__ in range(iteration) :
        '''要用Variable包裝tensor才能送入模型'''
        if Use_cuda:
            image_ = Variable(image_in.cuda())
            y = [Variable(torch.tensor(label_0).cuda(), volatile=True),Variable(torch.tensor(label_1).cuda(), 
                volatile=True),Variable(torch.tensor(label_2).cuda(), volatile=True),Variable(torch.tensor(label_3).cuda(), volatile=True)]      
        else:
            image_ = Variable(image_in)
            y = [Variable(torch.tensor(label_0), volatile=True),Variable(torch.tensor(label_1), 
                volatile=True),Variable(torch.tensor(label_2), volatile=True),Variable(torch.tensor(label_3), volatile=True)]

        '''Forward Pass'''
        out = net(image_)
        '''Regression Loss and Classification Loss'''
        loss_l,loss_c = criterion(out,y )
        loss = loss_l+ loss_c
        '''Backward'''
        loss.backward()

        loc_loss.append(loss_l.data.cpu().numpy())
        conf_loss.append(loss_c.data.cpu().numpy())
        loss_sum.append(loss.data.cpu().numpy())
        '''更新參數'''
        optimizer.step()
        '''清空Gradients'''
        optimizer.zero_grad()
        
        n+=1
        if n%10==0:
            print('BBOX Regression Loss: ', np.mean(loc_loss))
            print('Classification Loss: ', np.mean(conf_loss))
    '''儲存權重'''
    torch.save(ssd_net.state_dict(),'weights/Ｗeights.pth')

C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\ipykernel_launcher.py:14: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  
C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\ipykernel_launcher.py:15: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  from ipykernel import kernelapp as app
C:\Users\user\Anaconda3\envs\Tesseract\lib\site-packages\torch\nn\_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


BBOX Regression Loss:  2.5160591408058446
Classification Loss:  13.88321691442419
BBOX Regression Loss:  2.5159284238462094
Classification Loss:  13.781624235930266
BBOX Regression Loss:  2.5157978387526527
Classification Loss:  13.681254106686437
BBOX Regression Loss:  2.5156673148826316
Classification Loss:  13.585246050799332
BBOX Regression Loss:  2.5155367985478154
Classification Loss:  13.498094120732063
BBOX Regression Loss:  2.5154058432873385
Classification Loss:  13.416101074218748
BBOX Regression Loss:  2.5152747139098155
Classification Loss:  13.33774265511326
BBOX Regression Loss:  2.5151433838738333
Classification Loss:  13.262045104415327
BBOX Regression Loss:  2.5150118918085296
Classification Loss:  13.188275473009902
BBOX Regression Loss:  2.514880399350767
Classification Loss:  13.115818696198644
BBOX Regression Loss:  2.5147488346806277
Classification Loss:  13.044589377252342
BBOX Regression Loss:  2.514617602030436
Classification Loss:  12.979137966956621
BBOX Reg

BBOX Regression Loss:  2.1835996559990782
Classification Loss:  11.626915853994864
BBOX Regression Loss:  1.8389315993697555
Classification Loss:  10.612622748480902
BBOX Regression Loss:  1.8359996089228878
Classification Loss:  10.600928469057434
BBOX Regression Loss:  1.8330518086751304
Classification Loss:  10.589068490487561
BBOX Regression Loss:  1.8300860970108594
Classification Loss:  10.577025717276113
BBOX Regression Loss:  1.8270956675211587
Classification Loss:  10.564781448929399
BBOX Regression Loss:  1.8240790802755475
Classification Loss:  10.55232238769531
BBOX Regression Loss:  1.8210488637288411
Classification Loss:  10.539621956638559
BBOX Regression Loss:  1.818000148843836
Classification Loss:  10.526669495194046
BBOX Regression Loss:  1.814923954402468
Classification Loss:  10.513456393269355
BBOX Regression Loss:  1.8118163652773254
Classification Loss:  10.499957738805701
BBOX Regression Loss:  1.8086776707710244
Classification Loss:  10.486156394264915
BBOX Re

BBOX Regression Loss:  0.1773957315125613
Classification Loss:  2.3884130947173587
BBOX Regression Loss:  0.17583402354441643
Classification Loss:  2.375846328621182
BBOX Regression Loss:  0.17427376669866065
Classification Loss:  2.363598661352087
BBOX Regression Loss:  0.017081261895321033
Classification Loss:  1.0859891891479492
BBOX Regression Loss:  0.01588788187062299
Classification Loss:  1.0809403384173357
BBOX Regression Loss:  0.014916304158575738
Classification Loss:  1.0776437806494443
BBOX Regression Loss:  0.01404028642508719
Classification Loss:  1.0744322918079516
BBOX Regression Loss:  0.01318478343663392
Classification Loss:  1.071619354530617
BBOX Regression Loss:  0.012417266030370454
Classification Loss:  1.0690952053776495
BBOX Regression Loss:  0.011723515966897292
Classification Loss:  1.0656864115800808
BBOX Regression Loss:  0.011072014465376184
Classification Loss:  1.057825846142239
BBOX Regression Loss:  0.010443165113405928
Classification Loss:  1.05135386

Classification Loss:  0.7236720824482467
BBOX Regression Loss:  0.0002322345893869593
Classification Loss:  0.72364070073198
BBOX Regression Loss:  0.0002323195129851408
Classification Loss:  0.7236098205503613
BBOX Regression Loss:  0.00023269184077962912
Classification Loss:  0.7235791695811761
BBOX Regression Loss:  0.00023257467186568834
Classification Loss:  0.7235485906954164
BBOX Regression Loss:  0.0002327640003375249
Classification Loss:  0.7235188587117318
BBOX Regression Loss:  0.00023553399735154587
Classification Loss:  0.723489160364871
BBOX Regression Loss:  0.00023657305475658434
Classification Loss:  0.7234592077095373
BBOX Regression Loss:  0.0002366350066792766
Classification Loss:  0.7234298176971483
BBOX Regression Loss:  0.00023652653025129668
Classification Loss:  0.7234009373866225
BBOX Regression Loss:  0.00023632677978901466
Classification Loss:  0.7233723281462447
BBOX Regression Loss:  0.00023612249349067987
Classification Loss:  0.7233436742883205
BBOX Regr

...

Classification Loss:  0.4875558205776745
BBOX Regression Loss:  0.08500718768845257
Classification Loss:  0.48733219499941227
BBOX Regression Loss:  0.08534027916012388
Classification Loss:  0.4871127339458359
BBOX Regression Loss:  0.0854734044894782
Classification Loss:  0.486907476664512
BBOX Regression Loss:  0.0855044874880049
Classification Loss:  0.48671335088401574
BBOX Regression Loss:  0.08549909833548726
Classification Loss:  0.4865204516752267
BBOX Regression Loss:  0.08548660463756985
Classification Loss:  0.48632236435299825
BBOX Regression Loss:  0.08547808989672594
Classification Loss:  0.4861170408560326
BBOX Regression Loss:  0.085477421190513
Classification Loss:  0.48590565394962765
BBOX Regression Loss:  0.08550220168465471
Classification Loss:  0.48568940121531184
BBOX Regression Loss:  0.08570971937628241
Classification Loss:  0.48547525057444224
BBOX Regression Loss:  0.08578617325535527
Classification Loss:  0.48527008871384614
BBOX Regression Loss:  0.08580956

Classification Loss:  0.4138661234771612
BBOX Regression Loss:  0.09764432733491465
Classification Loss:  0.41367117402243747
BBOX Regression Loss:  0.09759924829742055
Classification Loss:  0.41346866843141156
BBOX Regression Loss:  0.09757409574917468
Classification Loss:  0.4132615493005453
BBOX Regression Loss:  0.0981361053608082
Classification Loss:  0.4130677813412611
BBOX Regression Loss:  0.09827176898034581
Classification Loss:  0.4129029059255915
BBOX Regression Loss:  0.09823999182769547
Classification Loss:  0.4127542527185546
BBOX Regression Loss:  0.09816663640177148
Classification Loss:  0.4126086596410159
BBOX Regression Loss:  0.09808418788224894
Classification Loss:  0.41245703889866075
BBOX Regression Loss:  0.09800647981526642
Classification Loss:  0.41229578084613555
BBOX Regression Loss:  0.09793728191348722
Classification Loss:  0.4121256110455216
BBOX Regression Loss:  0.09787821132395771
Classification Loss:  0.41194634071550756
BBOX Regression Loss:  0.097828

Classification Loss:  0.3473572917133994
BBOX Regression Loss:  0.09953021611640872
Classification Loss:  0.3471694589744157
BBOX Regression Loss:  0.09980069784797596
Classification Loss:  0.3469861027962223
BBOX Regression Loss:  0.09988806901154694
Classification Loss:  0.3468150056010545
BBOX Regression Loss:  0.09983672493349308
Classification Loss:  0.34665190839262866
BBOX Regression Loss:  0.09974161065035318
Classification Loss:  0.34648604142038447
BBOX Regression Loss:  0.0996473403878291
Classification Loss:  0.34631314027964083
BBOX Regression Loss:  0.09955661727883722
Classification Loss:  0.34613345745443475
BBOX Regression Loss:  0.09947970240204422
Classification Loss:  0.3459470698274212
BBOX Regression Loss:  0.09956194681330892
Classification Loss:  0.3457597995426847
BBOX Regression Loss:  0.09975407909748803
Classification Loss:  0.3455871417958249
BBOX Regression Loss:  0.09971727303096227
Classification Loss:  0.3454256700249997
BBOX Regression Loss:  0.0996332

Classification Loss:  0.2877532404977918
BBOX Regression Loss:  0.09543727075518164
Classification Loss:  0.28768584805271397
BBOX Regression Loss:  0.09519164615575179
Classification Loss:  0.28759688072734413
BBOX Regression Loss:  0.09496661785959026
Classification Loss:  0.2874857237189445
BBOX Regression Loss:  0.09476419835620456
Classification Loss:  0.2873546186376501
BBOX Regression Loss:  0.09458072729186902
Classification Loss:  0.2872086781562024
BBOX Regression Loss:  0.09441551951601294
Classification Loss:  0.28704918898068943
BBOX Regression Loss:  0.09426662881919173
Classification Loss:  0.2868778879870742
BBOX Regression Loss:  0.09413032346955706
Classification Loss:  0.2866988194822775
BBOX Regression Loss:  0.09400451496393993
Classification Loss:  0.28651378910950936
BBOX Regression Loss:  0.09388800755379693
Classification Loss:  0.28632431238416645
BBOX Regression Loss:  0.09378918840793761
Classification Loss:  0.2861303762308261
BBOX Regression Loss:  0.09452

BBOX Regression Loss:  0.08302429848247105
Classification Loss:  0.23335691487347637
BBOX Regression Loss:  0.08286344492446225
Classification Loss:  0.23318370820814785
BBOX Regression Loss:  0.08278056734243189
Classification Loss:  0.23300607557649966
BBOX Regression Loss:  0.08387925752168272
Classification Loss:  0.23284825736546083
BBOX Regression Loss:  0.08407344567253935
Classification Loss:  0.2327341587455184
BBOX Regression Loss:  0.08395118198277036
Classification Loss:  0.23263948903652867
BBOX Regression Loss:  0.08374824858901965
Classification Loss:  0.23254415297853778
BBOX Regression Loss:  0.08354338787032264
Classification Loss:  0.23243598708775778
BBOX Regression Loss:  0.08334634113642904
Classification Loss:  0.23231381921120633
BBOX Regression Loss:  0.08316418840771629
Classification Loss:  0.23218095120569113
BBOX Regression Loss:  0.08299926509680572
Classification Loss:  0.23203452671898736
BBOX Regression Loss:  0.08285223642119309
Classification Loss:  0

## 想要Train VOC,COCO可以參考原Github:https://github.com/amdegroot/ssd.pytorch